In [3]:
import pandas as pd


PATH = "G:\\Repos\\GT\\ISYE6420\\Final\\P3.dat"

df = pd.read_csv(PATH, sep="\t")

df.head()


df["aveinc"]    = df["aveinc"] / 10000
df["avehage"]   = df["avehage"] / 10
df["distcomp"]  = df["distcomp"] / 10
df["diststore"] = df["diststore"] / 10




,customers,hunits,aveinc,avehage,distcomp,diststore
0,9,606,41393,3,3.04,6.32
1,6,641,23635,18,1.95,8.89
2,28,505,55475,27,6.54,2.05
3,11,866,64646,31,1.67,5.81
4,4,599,31972,7,0.72,8.11


In [16]:
import numpy as np
import seaborn as sns
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.eval_measures import rmse

# 01: ('aveinc', 'avehage')
# 02: ('aveinc', 'distcomp')
# 03: ('aveinc', 'diststore')
# 04: ('avehage', 'distcomp')
# 05: ('avehage', 'diststore')
# 06: ('distcomp', 'diststore')
    

models = [
    ('aveinc', 'avehage'), ('aveinc', 'distcomp'), ('aveinc', 'diststore'),
    ('avehage', 'distcomp'), ('avehage', 'diststore'), ('distcomp', 'diststore'),
    ]

dsumm = {}
summlist = []
for indx, tt in enumerate(models):
    print(indx)
    desc = "mdl_{}".format(str(indx).zfill(2))
    params = ":".join(tt)
    expr = "customers ~ " + " + ".join(tt)
    mdl = smf.glm(
        formula=expr, data=df, 
        family=sm.families.Poisson(link=sm.families.links.log),
        exposure=df["hunits"]
        ).fit()

#     dmdl = {
#         "params":params, "null_deviance":mdl.null_deviance,
#         "deviance":mdl.deviance, "llnull":mdl.llnull, "ll":mdl.llf,
#         "aic":mdl.aic, "bic":mdl.bic, "pseudoR2":1 - (mdl.deviance / mdl.null_deviance),
#         "chi2":mdl.pearson_chi2,
#         }
    dmdl = {
        "params":params, "null_deviance":mdl.null_deviance, "deviance":mdl.deviance, 
        "ll":mdl.llf, "aic":mdl.aic, "pseudoR2":1 - (mdl.deviance / mdl.null_deviance),
        }
    
    dsumm[desc] = dmdl
    df[desc] = mdl.fittedvalues
    
    
# Convert list of dicts to DataFrame.
dfsumm = pd.DataFrame().from_dict(dsumm, orient="index")
dfsumm


0
1
2
3
4
5


,params,null_deviance,deviance,ll,aic,pseudoR2
mdl_00,aveinc:avehage,403.260504,498.087496,-457.246679,920.493359,-0.235151
mdl_01,aveinc:distcomp,403.260504,312.143607,-364.274735,734.549470,0.225950
mdl_02,aveinc:diststore,403.260504,284.790563,-350.598213,707.196426,0.293780
mdl_03,avehage:distcomp,403.260504,391.406387,-403.906125,813.812250,0.029396
mdl_04,avehage:diststore,403.260504,378.229523,-397.317693,800.635386,0.062071
mdl_05,distcomp:diststore,403.260504,370.793877,-393.599870,793.199740,0.080510


In [34]:
import numpy as np

ll = (-df["mdl_01"].values + df["customers"].values * np.log(df["mdl_01"].values)).sum()
aa = -2 * ll + 6
dd0 = 2 * (df["customers"].values * np.log((df["customers"].values / df["mdl_01"].values) + .01) - df["customers"].values - df["mdl_01"].values).sum()
dd1 = 2 * (df["customers"].values * np.log((df["customers"].values / df["mdl_01"].values) + .01)).sum()

print(dd0)
print(dd1)


-4336.828812673914
335.1711873260792


In [33]:
#df["mdl_01"].sum()
df["customers"].sum()

1168

In [35]:
np.clip?


In [28]:
sm.families.family?